In [1]:
%load_ext autoreload
%autoreload 2

from functools import partial

from transformer_lens import HookedTransformer

from eap.graph import Graph
from eap.evaluate import evaluate_graph, evaluate_baseline
from eap.attribute import attribute

from MIB_circuit_track.dataset import HFEAPDataset
from MIB_circuit_track.metrics import get_metric
from MIB_circuit_track.evaluation import evaluate_area_under_curve

In [2]:
model_name = "gpt2-small"
model = HookedTransformer.from_pretrained(model_name, device="cuda")
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True
model.cfg.ungroup_grouped_query_attention = True

Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
dataset = HFEAPDataset("mech-interp-bench/ioi", model.tokenizer, task="ioi", num_examples=100)
dataloader = dataset.to_dataloader(20)
metric_fn = get_metric("logit_diff", "ioi", model.tokenizer, model)

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
g = Graph.from_model(model)
attribute(model, g, dataloader, partial(metric_fn, loss=True, mean=True), 'EAP')

100%|██████████| 5/5 [00:03<00:00,  1.63it/s]


In [5]:
g.apply_topn(300, True)
baseline = evaluate_baseline(model, dataloader, partial(metric_fn, loss=False, mean=False)).mean().item()
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False)).mean().item()

print(f"Faithfulness: {results / baseline}. Original {baseline}, new {results}")

100%|██████████| 5/5 [00:01<00:00,  4.68it/s]

Faithfulness: 0.09384125455535235. Original 3.0810797214508057, new 0.28913238644599915


In [6]:
results = evaluate_area_under_curve(model, g, dataloader, partial(metric_fn, loss=False, mean=False))

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00,  5.18it/s]


Computing results for 0.1% of edges (N=32)


100%|██████████| 5/5 [00:00<00:00,  5.02it/s]


Computing results for 0.2% of edges (N=64)


...

100%|██████████| 5/5 [00:01<00:00,  5.00it/s]


Computing results for 50.0% of edges (N=16245)


100%|██████████| 5/5 [00:00<00:00,  5.10it/s]


Computing results for 100% of edges (N=32491)


100%|██████████| 5/5 [00:01<00:00,  2.54it/s]
